In [ ]:
import dash
from dash.dependencies import Input, Output
import json
from kafka import KafkaConsumer
import folium
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
import threading

# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Global variable to store truck data
truck_data = []

# Function to consume truck locations from Kafka in a separate thread
def consume_truck_data():
    global truck_data
    consumer = KafkaConsumer(
        'TruckLocations',
        bootstrap_servers=['localhost:29092'],
        value_deserializer=lambda v: json.loads(v.decode('utf-8')),
        auto_offset_reset='earliest'
    )
    
    for message in consumer:
        data = message.value
        #print(f"Received data: {data}")
        
        # Check if truck data already exists; if so, update it, else append
        for i, truck in enumerate(truck_data):
            if truck['truck_id'] == data['truck_id']:
                truck_data[i] = data
                break
        else:
            truck_data.append(data)

# Start the Kafka consumer in a separate thread
consumer_thread = threading.Thread(target=consume_truck_data)
consumer_thread.daemon = True
consumer_thread.start()

# Function to generate map HTML from truck data
def generate_map():
    # Initialize the map
    folium_map = folium.Map(location=[0, 0], zoom_start=2)
    
    # Add markers for each truck
    for data in truck_data:
        folium.Marker(
            [data['location']['latitude'], data['location']['longitude']],
            popup=f"Truck {data['truck_id']}"
        ).add_to(folium_map)
    
    # Save map to HTML file
    map_file = 'map.html'
    folium_map.save(map_file)
    
    # Read map HTML content
    with open(map_file, 'r') as file:
        map_html_content = file.read()
    
    return map_html_content

# Create Dash layout
app.layout = html.Div([
    html.H1("Live Truck Map"),
    dcc.Interval(
        id='interval-component',
        interval=1000,  # Update every 3 second
        n_intervals=0
    ),
    html.Div(id='map-container')
])

# Callback to update map
@app.callback(
    dash.dependencies.Output('map-container', 'children'),
    dash.dependencies.Input('interval-component', 'n_intervals')
)
def update_map(n):
    map_html_content = generate_map()
    
    # Convert map HTML to base64 to display in Dash
    map_html_base64 = base64.b64encode(map_html_content.encode()).decode()
    
    return html.Iframe(
        src=f'data:text/html;base64,{map_html_base64}',
        style={'width': '100%', 'height': '800px', 'border': 'none'}
    )

if __name__ == '__main__':
    app.run_server(debug=True)


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[3], line 85, in update_map(n=0)
     82 map_html_content = generate_map()
     84 # Convert map HTML to base64 to display in Dash
---> 85 map_html_base64 = base64.b64encode(map_html_content.encode()).decode()
        map_html_content = '<!DOCTYPE html>\n<html>\n<head>\n    \n    <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n    \n        <script>\n            L_NO_TOUCH = false;\n            L_DISABLE_3D = false;\n        </script>\n    \n    <style>html, body {width: 100%;height: 100%;margin: 0;padding: 0;}</style>\n    <style>#map {position:absolute;top:0;bottom:0;right:0;left:0;}</style>\n    <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>\n    <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>\n    <script src="https://cdn.jsdelivr.net/npm/bootstrap